# Build Education Index

## 1. Data Collection

- Number of educational services: from educational dataset

- Distance to centers: polygon or compute distances

- Region density over number of places offered: Oferta inicial de plazas escolares de los estudios obligatorios

- Ratio of students over teacher: Alumnos matriculados por enseñanza y unidades de centros docentes, Personal docente en centros públicos titularidad del Departamento de Educación, Conciertos educativos: unidad, alumnos, dotación de plantilla y gasto

- Academic Performance:

- School Dropout Rate:

- School Status - Pulic, Private or Charter: from educational dataset

- School Type Ponderation - School/Kindergarten, Library/Studying Room, Civic Center: from educational dataset

- Standarized Characteristics: international, religious, etc - dataset?

### Load Dataset with all Education Services

In [25]:
pip install pandas fuzzywuzzy[speedup]

Note: you may need to restart the kernel to use updated packages.


In [26]:
# IMPORTS
import pandas as pd
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

In [27]:
# Get rid of warnings
import warnings
warnings.filterwarnings("ignore")

In [28]:
import chardet

def detect_encoding(file_path):
    with open(file_path, 'rb') as f:
        result = chardet.detect(f.read())
    return result['encoding']

# Usage
#encoded = detect_encoding(file_name)
#print(encoded)

In [29]:
#from google.colab import drive
#drive.mount('/content/drive')

In [30]:
# Dataset Files
cleaned_educ_file = "/Users/mariatormo02/Desktop/TFG-Code/2.1 Data Processing/cleaned_education.csv"

encoded = detect_encoding(cleaned_educ_file)
print(encoded)

# Read CSV
cleaned_educ_df = pd.read_csv(cleaned_educ_file, encoding='utf-8')

utf-8


In [31]:
display(cleaned_educ_df)

,Unnamed: 0,Name,Road Name,Street Number,Neighborhood ID,Neighborhood Name,District ID,District Name,ZipCode,Service Type,geo_epgs_25831_x,geo_epgs_25831_y,Latitude,Longitude,Simple Service Type
0,0,Facultat d'Informació i Mitjans Audiovisuals - UB,Melcior de Palau,140.0,18,Sants,3,Sants-Montjuïc,8014,University,427989.387049,4.581443e+06,41.381286,2.138788,University
1,1,Institut Fort Pius,C Ausiàs Marc,78.0,5,el Fort Pienc,2,Eixample,8013,Secondary School,431433.012280,4.582861e+06,41.394359,2.179807,High School
2,2,Escola Pia de Nostra Senyora,Carrer de la Diputació,277.0,7,la Dreta de l'Eixample,2,Eixample,8007,Secondary School & FP,430435.525044,4.582521e+06,41.391216,2.167916,High School
3,3,Institut Salvador Seguí,C Santander,7.0,73,la Verneda i la Pau,10,Sant Martí,8020,FP,432882.376561,4.586038e+06,41.423094,2.196790,FP
4,4,Escola de Música Kostka,Rier Can Toda,29.0,30,la Salut,6,Gràcia,8024,Music School,429568.000936,4.585099e+06,41.414357,2.157240,Music School
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1352,1475,Biblioteca Trinitat Vella - José Barbero,NaN,NaN,57,la Trinitat Vella,9,Sant Andreu,0,Library & Study Room,NaN,NaN,41.449283,2.190749,Library & Study Room
1353,1479,Biblioteca Poblenou - Manuel Arranz,NaN,NaN,68,el Poblenou,10,Sant Martí,0,Library & Study Room,NaN,NaN,41.401373,2.204526,Library & Study Room
1354,1480,Biblioteca Ramon d'Alòs-Moner,NaN,NaN,70,el Besòs i el Maresme,10,Sant Martí,0,Library & Study Room,NaN,NaN,41.416309,2.211483,Library & Study Room
1355,1483,Biblioteca Poble Sec - Francesc Boix,NaN,NaN,11,el Poble Sec,3,Sants-Montjuïc,0,Library & Study Room,NaN,NaN,41.373922,2.164025,Library & Study Room


In [32]:
# Drop Unnecessary columns
cols_to_drop = ['Unnamed: 0', 'Road Name', 'Street Number', 'geo_epgs_25831_x', 'geo_epgs_25831_y']

cleaned_educ_df.drop(columns=cols_to_drop, inplace=True)
cleaned_educ_df.head(10)

,Name,Neighborhood ID,Neighborhood Name,District ID,District Name,ZipCode,Service Type,Latitude,Longitude,Simple Service Type
0,Facultat d'Informació i Mitjans Audiovisuals - UB,18,Sants,3,Sants-Montjuïc,8014,University,41.381286,2.138788,University
1,Institut Fort Pius,5,el Fort Pienc,2,Eixample,8013,Secondary School,41.394359,2.179807,High School
2,Escola Pia de Nostra Senyora,7,la Dreta de l'Eixample,2,Eixample,8007,Secondary School & FP,41.391216,2.167916,High School
3,Institut Salvador Seguí,73,la Verneda i la Pau,10,Sant Martí,8020,FP,41.423094,2.196790,FP
4,Escola de Música Kostka,30,la Salut,6,Gràcia,8024,Music School,41.414357,2.157240,Music School
5,Centre Educatiu Sagrat Cor-Diputació,7,la Dreta de l'Eixample,2,Eixample,8009,"Kindergarten, Primary & Secondary School",41.395298,2.174295,General School (with kindergarten)
6,Centre Educatiu Sant Francesc d'Assís,10,Sant Antoni,2,Eixample,8007,Kindergarten & Primary School,41.385181,2.163444,Kindergarten & Primary School
7,Centre Educatiu Sant Josep,31,la Vila de Gràcia,6,Gràcia,8012,"Kindergarten, Primary & Secondary School",41.402367,2.155367,General School (with kindergarten)
8,Escola Molí de Finestrelles,60,Sant Andreu,9,Sant Andreu,8030,Kindergarten & Primary School,41.438824,2.189559,Kindergarten & Primary School
9,Escola Casas,65,el Clot,10,Sant Martí,8018,Kindergarten & Primary School,41.408477,2.192800,Kindergarten & Primary School


In [33]:
# Move Simple Service Type column order
column_to_move = 'Simple Service Type'
new_position = 7 # start pos at 0

# Pop the col and insert at new position
column_pop = cleaned_educ_df.pop(column_to_move)
cleaned_educ_df.insert(new_position, column_to_move, column_pop)

cleaned_educ_df.head(10)

,Name,Neighborhood ID,Neighborhood Name,District ID,District Name,ZipCode,Service Type,Simple Service Type,Latitude,Longitude
0,Facultat d'Informació i Mitjans Audiovisuals - UB,18,Sants,3,Sants-Montjuïc,8014,University,University,41.381286,2.138788
1,Institut Fort Pius,5,el Fort Pienc,2,Eixample,8013,Secondary School,High School,41.394359,2.179807
2,Escola Pia de Nostra Senyora,7,la Dreta de l'Eixample,2,Eixample,8007,Secondary School & FP,High School,41.391216,2.167916
3,Institut Salvador Seguí,73,la Verneda i la Pau,10,Sant Martí,8020,FP,FP,41.423094,2.196790
4,Escola de Música Kostka,30,la Salut,6,Gràcia,8024,Music School,Music School,41.414357,2.157240
5,Centre Educatiu Sagrat Cor-Diputació,7,la Dreta de l'Eixample,2,Eixample,8009,"Kindergarten, Primary & Secondary School",General School (with kindergarten),41.395298,2.174295
6,Centre Educatiu Sant Francesc d'Assís,10,Sant Antoni,2,Eixample,8007,Kindergarten & Primary School,Kindergarten & Primary School,41.385181,2.163444
7,Centre Educatiu Sant Josep,31,la Vila de Gràcia,6,Gràcia,8012,"Kindergarten, Primary & Secondary School",General School (with kindergarten),41.402367,2.155367
8,Escola Molí de Finestrelles,60,Sant Andreu,9,Sant Andreu,8030,Kindergarten & Primary School,Kindergarten & Primary School,41.438824,2.189559
9,Escola Casas,65,el Clot,10,Sant Martí,8018,Kindergarten & Primary School,Kindergarten & Primary School,41.408477,2.192800


### 1.1 Create Dataframe for neighborhoods

In [34]:
print("Array of Neighborhood IDs:", cleaned_educ_df['Neighborhood ID'].unique())
print("Min Neighborhood ID:", min(cleaned_educ_df['Neighborhood ID'].unique()))
print("Min Neighborhood ID:", max(cleaned_educ_df['Neighborhood ID'].unique()))

Array of Neighborhood IDs: [18  5  7 73 30 10 31 60 65 13 71 24  1 35  9 45 72 68  8 22  4 23 63 52
  6 61 25 20 21 53 41 59 67 15 26 11 39 33 37 70 17  3 19 62 40 36 14 28
 51 27 43 48 66 64 44 12  2 16 47 46 55 58 29 50 34 32 56 42 57 69 49 54
  0 38]
Min Neighborhood ID: 0
Min Neighborhood ID: 73


In [35]:
# Zone IDs
neighborhood_ids = sorted(cleaned_educ_df['Neighborhood ID'].unique())

# Column Names
column_names = {
    0: 'Neighborhood Name',
    1: 'District Name Associated',
    2: 'Number of Services',
    3: 'Distances',
    4: 'Density over Places Ratio',
    5: 'Avg. Students per Teacher Ratio',
    6: 'School Status',
    7: 'Centers Type',
    8: 'Centers Simple Type',
    9: 'Centers Characteristics',
    10: 'Avg. Dropout Rate',
    11: 'Territorial Socioeconomic Index (IST)',
    12: 'Avg. Gini Index (%)',
    13: 'Population Density (hab/ha)',
    14: 'Net Population Density (hab/residential ha)',
}

# Create empty dataframe with IDs as index
zones_educ_df = pd.DataFrame(index=neighborhood_ids, columns=column_names.values())

# Rename Index
zones_educ_df.index.name = 'Neighborhood ID'

display(zones_educ_df)

,Neighborhood Name,District Name Associated,Number of Services,Distances,Density over Places Ratio,Avg. Students per Teacher Ratio,School Status,Centers Type,Centers Simple Type,Centers Characteristics,Avg. Dropout Rate,Territorial Socioeconomic Index (IST),Avg. Gini Index (%),Population Density (hab/ha),Net Population Density (hab/residential ha)
Neighborhood ID,,,,,,,,,,,,,,,
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
70,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
71,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### 1.2. Number of educational services

In [36]:
# Count number of rows for which Neighborhood ID holds a value
neighborhood_counts = cleaned_educ_df['Neighborhood ID'].value_counts().sort_index()
#print(neighborhood_counts)
zones_educ_df['Number of Services'] = neighborhood_counts
zones_educ_df.head(5)

,Neighborhood Name,District Name Associated,Number of Services,Distances,Density over Places Ratio,Avg. Students per Teacher Ratio,School Status,Centers Type,Centers Simple Type,Centers Characteristics,Avg. Dropout Rate,Territorial Socioeconomic Index (IST),Avg. Gini Index (%),Population Density (hab/ha),Net Population Density (hab/residential ha)
Neighborhood ID,,,,,,,,,,,,,,,
0,NaN,NaN,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,45,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,23,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,35,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### 1.3. Neighborhood & District Name Mappings

1.3.1 Neighborhoods

In [37]:
# Neighborhood ID & Name Mapping
# Mapping Generation (Unique IDs)
neighborhood_mapping = cleaned_educ_df[['Neighborhood ID', 'Neighborhood Name']].drop_duplicates()
#neighborhood_mapping = neighborhood_mapping.dropna()
#display(neighborhood_mapping[neighborhood_mapping['Neighborhood ID'] == 11]): idx 58 el Poble-sec, idx 1355 el Poble Sec
neighborhood_mapping = neighborhood_mapping.drop(1355)

# Set the index to 'Neighborhood ID'
neighborhood_mapping.set_index('Neighborhood ID', inplace=True)
# Sort IDs
neighborhood_mapping = neighborhood_mapping.sort_index()
#display(neighborhood_mapping)

# Merge Data
zones_educ_df['Neighborhood Name'] = neighborhood_mapping
zones_educ_df.head(5)

,Neighborhood Name,District Name Associated,Number of Services,Distances,Density over Places Ratio,Avg. Students per Teacher Ratio,School Status,Centers Type,Centers Simple Type,Centers Characteristics,Avg. Dropout Rate,Territorial Socioeconomic Index (IST),Avg. Gini Index (%),Population Density (hab/ha),Net Population Density (hab/residential ha)
Neighborhood ID,,,,,,,,,,,,,,,
0,NaN,NaN,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,el Raval,NaN,45,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,el Barri Gòtic,NaN,13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,la Barceloneta,NaN,23,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"Sant Pere, Santa Caterina i la Ribera",NaN,35,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [38]:
# Services with Neighborhood ID 0
#display(cleaned_educ_df[cleaned_educ_df['Neighborhood ID'] == 0])

# Drop ID 0
zones_educ_df = zones_educ_df.drop(zones_educ_df.index[0])
zones_educ_df.head(5)

,Neighborhood Name,District Name Associated,Number of Services,Distances,Density over Places Ratio,Avg. Students per Teacher Ratio,School Status,Centers Type,Centers Simple Type,Centers Characteristics,Avg. Dropout Rate,Territorial Socioeconomic Index (IST),Avg. Gini Index (%),Population Density (hab/ha),Net Population Density (hab/residential ha)
Neighborhood ID,,,,,,,,,,,,,,,
1,el Raval,NaN,45,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,el Barri Gòtic,NaN,13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,la Barceloneta,NaN,23,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"Sant Pere, Santa Caterina i la Ribera",NaN,35,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,el Fort Pienc,NaN,19,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


1.3.2 Districts

In [39]:
# Neighborhood ID & District Name Mapping
# Mapping Generation (Unique IDs)
district_mapping = cleaned_educ_df[['Neighborhood ID', 'District Name']].drop_duplicates()
#district_mapping = district_mapping[district_mapping['Neighborhood ID'] != 0]
district_mapping = district_mapping.dropna()
#print(district_mapping.isnull().any())

# Set the index to 'Neighborhood ID'
district_mapping.set_index('Neighborhood ID', inplace=True)

# Sort IDs
district_mapping = district_mapping.sort_index()
#display(district_mapping)

# Merge Data
zones_educ_df['District Name Associated'] = district_mapping
zones_educ_df.head(5)

,Neighborhood Name,District Name Associated,Number of Services,Distances,Density over Places Ratio,Avg. Students per Teacher Ratio,School Status,Centers Type,Centers Simple Type,Centers Characteristics,Avg. Dropout Rate,Territorial Socioeconomic Index (IST),Avg. Gini Index (%),Population Density (hab/ha),Net Population Density (hab/residential ha)
Neighborhood ID,,,,,,,,,,,,,,,
1,el Raval,Ciutat Vella,45,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,el Barri Gòtic,Ciutat Vella,13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,la Barceloneta,Ciutat Vella,23,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"Sant Pere, Santa Caterina i la Ribera",Ciutat Vella,35,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,el Fort Pienc,Eixample,19,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### 1.4. IST for all Neighborhoods

In [40]:
# Dataset Files
socioec_idx_file = "/Users/mariatormo02/Desktop/TFG-Code/Data/Context Index Data/IST.csv"

encoded = detect_encoding(socioec_idx_file)
print(encoded)

# Read CSV
socioec_idx_df = pd.read_csv(socioec_idx_file, encoding='utf-8', sep='\t')

utf-8


In [41]:
#display(socioec_idx_df)
# Only last year included
socioec_idx_df = socioec_idx_df[socioec_idx_df['any'] == 2020]

# Drop useless cols and renaming
socioec_idx_df = socioec_idx_df.drop(columns={'any', 'estat', 'concepte'})
socioec_idx_df.rename(columns={'barris de Barcelona': 'Neighborhood Name', 'valor': 'Territorial Socioeconomic Index (IST)'}, inplace=True)

# Reset indices
socioec_idx_df.reset_index(drop=True, inplace=True)
socioec_idx_df.index = socioec_idx_df.index + 1
socioec_idx_df.index.name = 'Neighborhood ID'
socioec_idx_df = socioec_idx_df.drop(socioec_idx_df.index[73])
display(socioec_idx_df)

,Neighborhood Name,Territorial Socioeconomic Index (IST)
Neighborhood ID,,
1,el Raval,"78,2"
2,el Barri Gòtic,"75,6"
3,la Barceloneta,"95,8"
4,"Sant Pere, Santa Caterina i la Ribera","100,8"
5,el Fort Pienc,111
...,...,...
69,Diagonal Mar i el Front Marítim del Poblenou,"116,6"
70,el Besòs i el Maresme,"72,4"
71,Provençals del Poblenou,107


In [42]:
# Compare
#socioec_idx_df['Neighborhood Name'].unique()
#zones_educ_df['Neighborhood Name'].unique()

# Merge
zones_educ_df['Territorial Socioeconomic Index (IST)'] = socioec_idx_df['Territorial Socioeconomic Index (IST)']
zones_educ_df.head(5)

,Neighborhood Name,District Name Associated,Number of Services,Distances,Density over Places Ratio,Avg. Students per Teacher Ratio,School Status,Centers Type,Centers Simple Type,Centers Characteristics,Avg. Dropout Rate,Territorial Socioeconomic Index (IST),Avg. Gini Index (%),Population Density (hab/ha),Net Population Density (hab/residential ha)
Neighborhood ID,,,,,,,,,,,,,,,
1,el Raval,Ciutat Vella,45,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"78,2",NaN,NaN,NaN
2,el Barri Gòtic,Ciutat Vella,13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"75,6",NaN,NaN,NaN
3,la Barceloneta,Ciutat Vella,23,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"95,8",NaN,NaN,NaN
4,"Sant Pere, Santa Caterina i la Ribera",Ciutat Vella,35,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"100,8",NaN,NaN,NaN
5,el Fort Pienc,Eixample,19,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,111,NaN,NaN,NaN


In [43]:
# Replace COMA with POINT for decimals in the "Territorial Socioeconomic Index (IST)" column
zones_educ_df['Territorial Socioeconomic Index (IST)'] = zones_educ_df['Territorial Socioeconomic Index (IST)'].str.replace(',', '.')

# Convert the column to numeric data type
zones_educ_df['Territorial Socioeconomic Index (IST)'] = pd.to_numeric(zones_educ_df['Territorial Socioeconomic Index (IST)'])
zones_educ_df.head(5)

,Neighborhood Name,District Name Associated,Number of Services,Distances,Density over Places Ratio,Avg. Students per Teacher Ratio,School Status,Centers Type,Centers Simple Type,Centers Characteristics,Avg. Dropout Rate,Territorial Socioeconomic Index (IST),Avg. Gini Index (%),Population Density (hab/ha),Net Population Density (hab/residential ha)
Neighborhood ID,,,,,,,,,,,,,,,
1,el Raval,Ciutat Vella,45,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,78.2,NaN,NaN,NaN
2,el Barri Gòtic,Ciutat Vella,13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,75.6,NaN,NaN,NaN
3,la Barceloneta,Ciutat Vella,23,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,95.8,NaN,NaN,NaN
4,"Sant Pere, Santa Caterina i la Ribera",Ciutat Vella,35,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,100.8,NaN,NaN,NaN
5,el Fort Pienc,Eixample,19,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,111.0,NaN,NaN,NaN


### 1.5. Gini Index(%) for all Neighborhoods

In [45]:
# Dataset Files
gini_idx_file = "/Users/mariatormo02/Desktop/TFG-Code/Data/Context Index Data/2021_renda_index_gini.csv"

encoded = detect_encoding(gini_idx_file)
print(encoded)

# Read CSV
gini_idx_df = pd.read_csv(gini_idx_file, encoding='utf-8')

utf-8


In [46]:
display(gini_idx_df)

,Any,Codi_Districte,Nom_Districte,Codi_Barri,Nom_Barri,Seccio_Censal,Index_Gini
0,2021,1,Ciutat Vella,1,el Raval,1,37.5
1,2021,1,Ciutat Vella,1,el Raval,2,35.4
2,2021,1,Ciutat Vella,1,el Raval,3,36.2
3,2021,1,Ciutat Vella,1,el Raval,4,35.0
4,2021,1,Ciutat Vella,1,el Raval,5,36.8
...,...,...,...,...,...,...,...
1063,2021,10,Sant Martí,73,la Verneda i la Pau,143,27.1
1064,2021,10,Sant Martí,65,el Clot,234,25.3
1065,2021,10,Sant Martí,69,Diagonal Mar i el Front Marítim del Poblenou,235,32.9
1066,2021,10,Sant Martí,69,Diagonal Mar i el Front Marítim del Poblenou,236,28.1


In [47]:
#display(socioec_idx_df)
# Drop useless cols and renaming
gini_idx_df = gini_idx_df.drop(columns={'Any', 'Codi_Districte', 'Nom_Districte'})
gini_idx_df.rename(columns={'Codi_Barri': 'Neighborhood ID', 'Nom_Barri': 'Neighborhood Name', 'Index_Gini': 'Gini Index'}, inplace=True)

display(gini_idx_df)

,Neighborhood ID,Neighborhood Name,Seccio_Censal,Gini Index
0,1,el Raval,1,37.5
1,1,el Raval,2,35.4
2,1,el Raval,3,36.2
3,1,el Raval,4,35.0
4,1,el Raval,5,36.8
...,...,...,...,...
1063,73,la Verneda i la Pau,143,27.1
1064,65,el Clot,234,25.3
1065,69,Diagonal Mar i el Front Marítim del Poblenou,235,32.9
1066,69,Diagonal Mar i el Front Marítim del Poblenou,236,28.1


In [48]:
# Compute Avg Gini Index per Neighborhood ID
avg_gini_df = gini_idx_df.groupby('Neighborhood ID')['Gini Index'].mean()

# Into sorted Dataframe by ID
avg_gini_df = avg_gini_df.reset_index()
avg_gini_df.index = avg_gini_df.index + 1
avg_gini_df.rename(columns={'Gini Index': 'Avg. Gini Index (%)'}, inplace=True)
avg_gini_df.drop(columns={'Neighborhood ID'}, inplace=True)
display(avg_gini_df)

,Avg. Gini Index (%)
1,34.495238
2,39.133333
3,33.636364
4,37.030769
5,32.475000
...,...
69,31.814286
70,29.715385
71,29.241667
72,27.361111


In [49]:
# Merge into zones dataset (round to 3 decimal places)
avg_gini_df['Avg. Gini Index (%)'] = avg_gini_df['Avg. Gini Index (%)'].round(3)
zones_educ_df['Avg. Gini Index (%)'] = avg_gini_df['Avg. Gini Index (%)']
display(zones_educ_df)

,Neighborhood Name,District Name Associated,Number of Services,Distances,Density over Places Ratio,Avg. Students per Teacher Ratio,School Status,Centers Type,Centers Simple Type,Centers Characteristics,Avg. Dropout Rate,Territorial Socioeconomic Index (IST),Avg. Gini Index (%),Population Density (hab/ha),Net Population Density (hab/residential ha)
Neighborhood ID,,,,,,,,,,,,,,,
1,el Raval,Ciutat Vella,45,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,78.2,34.495,NaN,NaN
2,el Barri Gòtic,Ciutat Vella,13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,75.6,39.133,NaN,NaN
3,la Barceloneta,Ciutat Vella,23,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,95.8,33.636,NaN,NaN
4,"Sant Pere, Santa Caterina i la Ribera",Ciutat Vella,35,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,100.8,37.031,NaN,NaN
5,el Fort Pienc,Eixample,19,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,111.0,32.475,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69,Diagonal Mar i el Front Marítim del Poblenou,Sant Martí,7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,116.6,31.814,NaN,NaN
70,el Besòs i el Maresme,Sant Martí,13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,72.4,29.715,NaN,NaN
71,Provençals del Poblenou,Sant Martí,17,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,107.0,29.242,NaN,NaN


### 1.6. Residents Density for each Neighborhood

In [50]:
# Dataset Files
density_file = "/Users/mariatormo02/Desktop/TFG-Code/Data/Context Index Data/2021_densitat.csv"

encoded = detect_encoding(density_file)
print(encoded)

# Read CSV
density_df = pd.read_csv(density_file, encoding='utf-8')

utf-8


In [51]:
display(density_df)

,Any,Codi_Districte,Nom_Districte,Codi_Barri,Nom_Barri,Població,Superfície (ha),Superfície Residencial (ha),Densitat (hab/ha),Densitat neta (hab/ha)
0,2021,1,Ciutat Vella,1,el Raval,47228,110.0,49.6,429,952
1,2021,1,Ciutat Vella,2,el Barri Gòtic,21827,81.6,34.1,268,639
2,2021,1,Ciutat Vella,3,la Barceloneta,14643,117.9,13.5,124,1081
3,2021,1,Ciutat Vella,4,"Sant Pere, Santa Caterina i la Ribera",22131,111.0,32.4,199,683
4,2021,2,Eixample,5,el Fort Pienc,31262,92.9,33.6,336,930
...,...,...,...,...,...,...,...,...,...,...
68,2021,10,Sant Martí,69,Diagonal Mar i el Front Marítim del Poblenou,13323,122.7,24.0,109,556
69,2021,10,Sant Martí,70,el Besòs i el Maresme,25657,120.9,28.1,212,913
70,2021,10,Sant Martí,71,Provençals del Poblenou,20949,108.6,15.3,193,1368
71,2021,10,Sant Martí,72,Sant Martí de Provençals,26032,73.4,23.4,355,1111


In [52]:
#display(density_df)
# Drop useless cols and renaming
density_df = density_df.drop(columns={'Any', 'Codi_Districte', 'Nom_Districte', 'Nom_Barri', 'Població', 'Superfície (ha)', 'Superfície Residencial (ha)'})
density_df.rename(columns={'Codi_Barri': 'Neighborhood ID', 'Densitat (hab/ha)': 'Population Density (hab/ha)', 'Densitat neta (hab/ha)': 'Net Population Density (hab/residential ha)'}, inplace=True)

# Use Index as Neighborhood ID
density_df.drop(columns={'Neighborhood ID'}, inplace=True)
density_df.index = density_df.index + 1
display(density_df)

,Population Density (hab/ha),Net Population Density (hab/residential ha)
1,429,952
2,268,639
3,124,1081
4,199,683
5,336,930
...,...,...
69,109,556
70,212,913
71,193,1368
72,355,1111


In [53]:
# Merge data
zones_educ_df['Population Density (hab/ha)'] = density_df['Population Density (hab/ha)']
zones_educ_df['Net Population Density (hab/residential ha)'] = density_df['Net Population Density (hab/residential ha)']
zones_educ_df.head(7)

,Neighborhood Name,District Name Associated,Number of Services,Distances,Density over Places Ratio,Avg. Students per Teacher Ratio,School Status,Centers Type,Centers Simple Type,Centers Characteristics,Avg. Dropout Rate,Territorial Socioeconomic Index (IST),Avg. Gini Index (%),Population Density (hab/ha),Net Population Density (hab/residential ha)
Neighborhood ID,,,,,,,,,,,,,,,
1,el Raval,Ciutat Vella,45,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,78.2,34.495,429,952
2,el Barri Gòtic,Ciutat Vella,13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,75.6,39.133,268,639
3,la Barceloneta,Ciutat Vella,23,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,95.8,33.636,124,1081
4,"Sant Pere, Santa Caterina i la Ribera",Ciutat Vella,35,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,100.8,37.031,199,683
5,el Fort Pienc,Eixample,19,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,111.0,32.475,336,930
6,la Sagrada Família,Eixample,24,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,111.8,32.700,493,1007
7,la Dreta de l'Eixample,Eixample,75,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,120.9,36.697,211,392


### 1.7 Services Status

In [55]:
# Dataset Files
status_file = "status_education.csv"

encoded = detect_encoding(status_file)
print(encoded)

# Read CSV
status_df = pd.read_csv(status_file, encoding='utf-8')

utf-8


In [56]:
status_df.drop(columns={'Unnamed: 0'}, inplace=True)
status_df.rename(columns={'Neighborhood Name': 'District Name'}, inplace=True)
display(status_df)

,Name,District Name,Status
0,Escola Poeta Foix,Sarrià-Sant Gervasi,public
1,Escola Arc Iris,Horta-Guinardó,public
2,Escola Baixeras,Ciutat Vella,public
3,Escola Parc de la Ciutadella,Ciutat Vella,public
4,Escola Mossèn Jacint Verdaguer,Sants-Montjuïc,public
...,...,...,...
549,Llar d'infants Guinyolet,la Sagrera,private
550,Llar d'infants La Pau - Gran Via,Sants-Montjuïc,private
551,Llar d'infants Àgora,Eixample,private
552,Escola Infantil Pas A Pas,Horta-Guinardó,private


In [57]:
# Function to get the best match with a minimum score threshold and district matching
def get_best_match(row, choices, scorer, threshold=95):
    # Filter choices by matching district
    district_choices = [choice for choice in choices if choice[1] == row['District Name']]

    if not district_choices:
        return None

    best_match, best_score = process.extractOne(row['Name'], [choice[0] for choice in district_choices], scorer=scorer)

    if best_score >= threshold:
        return best_match
    else:
        return None

In [58]:
# Assign Neighborhoods using Cleaned Dataset

# Assuming 'District Name' column exists in both datasets
# Prepare the choices list with tuples of (Name, District)
choices = list(cleaned_educ_df[['Name', 'District Name']].itertuples(index=False, name=None))

# Apply the fuzzy matching function to the second dataset
status_df['Matched_Name'] = status_df.apply(get_best_match, axis=1, choices=choices, scorer=fuzz.partial_ratio)

# Filter out rows where no good match was found
status_df = status_df[status_df['Matched_Name'].notna()]

# Merge with the neighborhood information using the matched names
status_merged_df = status_df.merge(cleaned_educ_df[['Name', 'Neighborhood Name']],
                                   left_on='Matched_Name', right_on='Name', how='left')

# Drop the 'Matched_Name' column as it is no longer needed
status_merged_df.drop(columns=['Matched_Name'], inplace=True)

# Rename and drop columns
status_merged_df.rename(columns={'Name_x': 'Original Name', 'Name_y': 'Mapped Name', 'Neighborhood Name_y': 'Neighborhood Name'}, inplace=True)

# Display the merged DataFrame
display(status_merged_df)
status_df = status_merged_df

,Original Name,District Name,Status,Mapped Name,Neighborhood Name
0,Escola Poeta Foix,Sarrià-Sant Gervasi,public,Escola Poeta Foix,el Putxet i el Farró
1,Escola Arc Iris,Horta-Guinardó,public,Escola Arc Iris,la Font d'en Fargues
2,Escola Baixeras,Ciutat Vella,public,Escola Baixeras,el Barri Gòtic
3,Escola Parc de la Ciutadella,Ciutat Vella,public,Escola Parc de la Ciutadella,"Sant Pere, Santa Caterina i la Ribera"
4,Escola Mossèn Jacint Verdaguer,Sants-Montjuïc,public,Escola Mossèn Jacint Verdaguer,el Poble-sec
...,...,...,...,...,...
389,Llar d'infants El petit rossinyol,Eixample,private,Llar d'Infants El Petit Rossinyol,Sant Antoni
390,Llar d'infants Tagore,Sarrià-Sant Gervasi,private,Llar d'Infants Tagore,Sant Gervasi - Galvany
391,Llar d'infants Casa Sagrada Família,Sarrià-Sant Gervasi,private,Llar d'Infants Casa Sagrada Família,"Vallvidrera, el Tibidabo i les Planes"
392,Llar d'infants La Pau - Gran Via,Sants-Montjuïc,private,Llar d'Infants La Pau,la Font de la Guatlla


In [59]:
# Group by Neighborhood, and count occurences of each status value

# Group by 'Neighborhood Name' and aggregate 'Status' column
neighborhood_status_df = status_df.groupby('Neighborhood Name')['Status'].value_counts().unstack(fill_value=0)

# Reset index to make 'Neighborhood Name' a column instead of an index
neighborhood_status_df.reset_index(inplace=True)

# Reorder columns
neighborhood_status_df = neighborhood_status_df[['Neighborhood Name', 'private', 'public', 'charter']]

display(neighborhood_status_df)

Status,Neighborhood Name,private,public,charter
0,Baró de Viver,0,2,1
1,Canyelles,0,3,0
2,Ciutat Meridiana,0,4,1
3,Diagonal Mar i el Front Marítim del Poblenou,0,1,2
4,Horta,0,3,5
...,...,...,...,...
59,la Vila Olímpica del Poblenou,0,2,0
60,la Vila de Gràcia,0,9,4
61,les Corts,0,4,3
62,les Roquetes,0,4,0


In [60]:
# Group by District, and count occurences of each status value

# Group by 'District Name' and aggregate 'Status' column
district_status_df = status_df.groupby('District Name')['Status'].value_counts().unstack(fill_value=0)

# Reset index to make 'Neighborhood Name' a column instead of an index
district_status_df.reset_index(inplace=True)

# Reorder columns
district_status_df = district_status_df[['District Name', 'private', 'public', 'charter']]

display(district_status_df)

Status,District Name,private,public,charter
0,Ciutat Vella,0,17,6
1,Eixample,5,22,22
2,Gràcia,0,22,11
3,Horta-Guinardó,1,28,13
4,Les Corts,1,11,9
5,Nou Barris,0,29,13
6,Sant Andreu,0,23,12
7,Sant Martí,0,36,18
8,Sants-Montjuïc,5,30,12
9,Sarrià-Sant Gervasi,3,13,32


We will assign the status offer regarding the Neighbourhood. In case some neighborhoods are not referenced but, instead, the District is given, we will assign status according to the district name instead of the neighborhood.

In [61]:
cleaned_educ_df['Neighborhood Name'].unique()

array(['Sants', 'el Fort Pienc', "la Dreta de l'Eixample",
       'la Verneda i la Pau', 'la Salut', 'Sant Antoni',
       'la Vila de Gràcia', 'Sant Andreu', 'el Clot', 'la Marina de Port',
       'Provençals del Poblenou', 'les Tres Torres', 'el Raval',
       'el Guinardó', "la Nova Esquerra de l'Eixample", 'Porta',
       'Sant Martí de Provençals', 'el Poblenou',
       "l'Antiga Esquerra de l'Eixample",
       'Vallvidrera, el Tibidabo i les Planes',
       'Sant Pere, Santa Caterina i la Ribera', 'Sarrià', 'Navas',
       'la Prosperitat', 'la Sagrada Família', 'la Sagrera',
       'Sant Gervasi - la Bonanova', 'la Maternitat i Sant Ramon',
       'Pedralbes', 'la Trinitat Nova', "la Vall d'Hebron",
       'el Bon Pastor', 'la Vila Olímpica del Poblenou', 'Hostafrancs',
       'Sant Gervasi - Galvany', 'el Poble-sec',
       'Sant Genís dels Agudells', 'el Baix Guinardó', 'el Carmel',
       'el Besòs i el Maresme', 'Sants - Badal', 'la Barceloneta',
       'les Corts', 'el Cong

In [62]:
#cleaned_educ_df['District Name'].unique()

In [63]:
neighborhood_status_df['Neighborhood Name'].unique()

array(['Baró de Viver', 'Canyelles', 'Ciutat Meridiana',
       'Diagonal Mar i el Front Marítim del Poblenou', 'Horta',
       'Hostafrancs', 'Montbau', 'Navas', 'Pedralbes', 'Porta',
       'Provençals del Poblenou', 'Sant Andreu', 'Sant Antoni',
       'Sant Genís dels Agudells', 'Sant Gervasi - Galvany',
       'Sant Gervasi - la Bonanova', 'Sant Martí de Provençals',
       'Sant Pere, Santa Caterina i la Ribera', 'Sants', 'Sants - Badal',
       'Sarrià', 'Vallbona', 'Vallcarca i els Penitents',
       'Vallvidrera, el Tibidabo i les Planes', 'Verdun',
       'Vilapicina i la Torre Llobeta', 'el Baix Guinardó',
       'el Barri Gòtic', 'el Besòs i el Maresme', 'el Bon Pastor',
       "el Camp d'en Grassot i Gràcia Nova", "el Camp de l'Arpa del Clot",
       'el Carmel', 'el Clot', 'el Coll', 'el Congrés i els Indians',
       'el Fort Pienc', 'el Guinardó', 'el Poble-sec', 'el Poblenou',
       'el Putxet i el Farró', 'el Raval', 'el Turó de la Peira',
       "l'Antiga Esquerra d

In [64]:
# Missing Neighborhoods
print("Missing Neighborhoods:", list(set(cleaned_educ_df['Neighborhood Name'].unique()) - set(status_df['Neighborhood Name'].unique())))

Missing Neighborhoods: [nan, 'la Marina del Prat Vermell', 'la Clota', 'el Parc i la Llacuna del Poblenou', 'la Trinitat Vella', 'la Teixonera', 'Can Peguera', 'el Poble Sec', 'Can Baró', 'la Trinitat Nova', 'Torre Baró']


In [65]:
# DISTRICTS NAMES

# Merge in case Name coincides, assign list of status greater than 0
# Create empty list
status_list = []

# Iterate over 'zones_educ_df' rows
for idx, row in zones_educ_df.iterrows():
  district_name = row['District Name Associated']

  # Check if in status dataframe
  if district_name in district_status_df['District Name'].values:
    # Get row
    district_row = district_status_df.loc[district_status_df['District Name'] == district_name]
    # Get status > 0
    status = [col for col in district_row.columns[1:] if district_row[col].values[0] > 0]

    # Append to list of statuses
    status_string = ', '.join(status)
    status_list.append(status_string)
  else:
    status_list.append('')

# Assign list
zones_educ_df['School Status'] = status_list
zones_educ_df.head(5)
#display(zones_educ_df)

,Neighborhood Name,District Name Associated,Number of Services,Distances,Density over Places Ratio,Avg. Students per Teacher Ratio,School Status,Centers Type,Centers Simple Type,Centers Characteristics,Avg. Dropout Rate,Territorial Socioeconomic Index (IST),Avg. Gini Index (%),Population Density (hab/ha),Net Population Density (hab/residential ha)
Neighborhood ID,,,,,,,,,,,,,,,
1,el Raval,Ciutat Vella,45,NaN,NaN,NaN,"public, charter",NaN,NaN,NaN,NaN,78.2,34.495,429,952
2,el Barri Gòtic,Ciutat Vella,13,NaN,NaN,NaN,"public, charter",NaN,NaN,NaN,NaN,75.6,39.133,268,639
3,la Barceloneta,Ciutat Vella,23,NaN,NaN,NaN,"public, charter",NaN,NaN,NaN,NaN,95.8,33.636,124,1081
4,"Sant Pere, Santa Caterina i la Ribera",Ciutat Vella,35,NaN,NaN,NaN,"public, charter",NaN,NaN,NaN,NaN,100.8,37.031,199,683
5,el Fort Pienc,Eixample,19,NaN,NaN,NaN,"private, public, charter",NaN,NaN,NaN,NaN,111.0,32.475,336,930


In [66]:
# NEIGHBORHOODS NAMES

# Merge in case Name coincides, assign list of status greater than 0
# Create empty list
status_list = []

# Iterate over 'zones_educ_df' rows
for idx, row in zones_educ_df.iterrows():
  neighborhood_name = row['Neighborhood Name']

  # Check if in status dataframe
  if neighborhood_name in neighborhood_status_df['Neighborhood Name'].values:
    # Get row
    neighborhood_row = neighborhood_status_df.loc[neighborhood_status_df['Neighborhood Name'] == neighborhood_name]
    # Get status > 0
    status = [col for col in neighborhood_row.columns[1:] if neighborhood_row[col].values[0] > 0]

    # Append to list of statuses
    status_string = ', '.join(status)
    status_list.append(status_string)
  else:
    status_list.append(row['School Status'])

# Assign list
zones_educ_df['School Status'] = status_list
zones_educ_df.head(5)
#display(zones_educ_df)

,Neighborhood Name,District Name Associated,Number of Services,Distances,Density over Places Ratio,Avg. Students per Teacher Ratio,School Status,Centers Type,Centers Simple Type,Centers Characteristics,Avg. Dropout Rate,Territorial Socioeconomic Index (IST),Avg. Gini Index (%),Population Density (hab/ha),Net Population Density (hab/residential ha)
Neighborhood ID,,,,,,,,,,,,,,,
1,el Raval,Ciutat Vella,45,NaN,NaN,NaN,"public, charter",NaN,NaN,NaN,NaN,78.2,34.495,429,952
2,el Barri Gòtic,Ciutat Vella,13,NaN,NaN,NaN,"public, charter",NaN,NaN,NaN,NaN,75.6,39.133,268,639
3,la Barceloneta,Ciutat Vella,23,NaN,NaN,NaN,public,NaN,NaN,NaN,NaN,95.8,33.636,124,1081
4,"Sant Pere, Santa Caterina i la Ribera",Ciutat Vella,35,NaN,NaN,NaN,"public, charter",NaN,NaN,NaN,NaN,100.8,37.031,199,683
5,el Fort Pienc,Eixample,19,NaN,NaN,NaN,"private, public, charter",NaN,NaN,NaN,NaN,111.0,32.475,336,930


In [67]:
zones_educ_df['School Status'].unique()

array(['public, charter', 'public', 'private, public, charter',
       'private, public', 'charter', 'private, charter'], dtype=object)

### 1.8 Service Typology (Service Type & Simple Service Type)

In [68]:
display(cleaned_educ_df)

,Name,Neighborhood ID,Neighborhood Name,District ID,District Name,ZipCode,Service Type,Simple Service Type,Latitude,Longitude
0,Facultat d'Informació i Mitjans Audiovisuals - UB,18,Sants,3,Sants-Montjuïc,8014,University,University,41.381286,2.138788
1,Institut Fort Pius,5,el Fort Pienc,2,Eixample,8013,Secondary School,High School,41.394359,2.179807
2,Escola Pia de Nostra Senyora,7,la Dreta de l'Eixample,2,Eixample,8007,Secondary School & FP,High School,41.391216,2.167916
3,Institut Salvador Seguí,73,la Verneda i la Pau,10,Sant Martí,8020,FP,FP,41.423094,2.196790
4,Escola de Música Kostka,30,la Salut,6,Gràcia,8024,Music School,Music School,41.414357,2.157240
...,...,...,...,...,...,...,...,...,...,...
1352,Biblioteca Trinitat Vella - José Barbero,57,la Trinitat Vella,9,Sant Andreu,0,Library & Study Room,Library & Study Room,41.449283,2.190749
1353,Biblioteca Poblenou - Manuel Arranz,68,el Poblenou,10,Sant Martí,0,Library & Study Room,Library & Study Room,41.401373,2.204526
1354,Biblioteca Ramon d'Alòs-Moner,70,el Besòs i el Maresme,10,Sant Martí,0,Library & Study Room,Library & Study Room,41.416309,2.211483
1355,Biblioteca Poble Sec - Francesc Boix,11,el Poble Sec,3,Sants-Montjuïc,0,Library & Study Room,Library & Study Room,41.373922,2.164025


In [69]:
# SERVICE TYPE
# Group by 'Neighborhood ID', include unique values for 'Service Type'
neighborhood_type_df = cleaned_educ_df.groupby('Neighborhood ID')['Service Type'].apply(lambda x: ', '.join(x.unique()))
# Reset idx values
neighborhood_type_df = neighborhood_type_df.reset_index()
neighborhood_type_df = neighborhood_type_df[neighborhood_type_df['Neighborhood ID'] != 0]

display(neighborhood_type_df)

,Neighborhood ID,Service Type
1,1,"Kindergarten & Primary School, University, Kin..."
2,2,"FP, Kindergarten & Primary School, University,..."
3,3,"Secondary School, University, Kindergarten & P..."
4,4,"Kindergarten & Primary School, Secondary Schoo..."
5,5,"Secondary School, Kindergarten & Primary Schoo..."
...,...,...
69,69,"Kindergarten, Primary & Secondary School, FP, ..."
70,70,"Kindergarten & Primary School, Secondary Schoo..."
71,71,"Kindergarten & Primary School, University, Inf..."
72,72,"Kindergarten & Primary School, Kindergarten, P..."


In [70]:
# SIMPLE SERVICE TYPE
# Group by 'Neighborhood ID', include unique values for 'Service Type'
neighborhood_simple_type_df = cleaned_educ_df.groupby('Neighborhood ID')['Simple Service Type'].apply(lambda x: ', '.join(x.unique()))
# Reset idx values
neighborhood_simple_type_df = neighborhood_simple_type_df.reset_index()
neighborhood_simple_type_df = neighborhood_simple_type_df[neighborhood_simple_type_df['Neighborhood ID'] != 0]

display(neighborhood_simple_type_df)

,Neighborhood ID,Simple Service Type
1,1,"Kindergarten & Primary School, University, Gen..."
2,2,"FP, Kindergarten & Primary School, University,..."
3,3,"High School, University, Kindergarten & Primar..."
4,4,"Kindergarten & Primary School, High School, Mu..."
5,5,"High School, Kindergarten & Primary School, Ge..."
...,...,...
69,69,"General School (with kindergarten), FP, Kinder..."
70,70,"Kindergarten & Primary School, High School, Un..."
71,71,"Kindergarten & Primary School, University, Inf..."
72,72,"Kindergarten & Primary School, General School ..."


In [71]:
zones_educ_df['Centers Type'] = neighborhood_type_df['Service Type']
zones_educ_df['Centers Simple Type'] = neighborhood_simple_type_df['Simple Service Type']
zones_educ_df.head(7)

,Neighborhood Name,District Name Associated,Number of Services,Distances,Density over Places Ratio,Avg. Students per Teacher Ratio,School Status,Centers Type,Centers Simple Type,Centers Characteristics,Avg. Dropout Rate,Territorial Socioeconomic Index (IST),Avg. Gini Index (%),Population Density (hab/ha),Net Population Density (hab/residential ha)
Neighborhood ID,,,,,,,,,,,,,,,
1,el Raval,Ciutat Vella,45,NaN,NaN,NaN,"public, charter","Kindergarten & Primary School, University, Kin...","Kindergarten & Primary School, University, Gen...",NaN,NaN,78.2,34.495,429,952
2,el Barri Gòtic,Ciutat Vella,13,NaN,NaN,NaN,"public, charter","FP, Kindergarten & Primary School, University,...","FP, Kindergarten & Primary School, University,...",NaN,NaN,75.6,39.133,268,639
3,la Barceloneta,Ciutat Vella,23,NaN,NaN,NaN,public,"Secondary School, University, Kindergarten & P...","High School, University, Kindergarten & Primar...",NaN,NaN,95.8,33.636,124,1081
4,"Sant Pere, Santa Caterina i la Ribera",Ciutat Vella,35,NaN,NaN,NaN,"public, charter","Kindergarten & Primary School, Secondary Schoo...","Kindergarten & Primary School, High School, Mu...",NaN,NaN,100.8,37.031,199,683
5,el Fort Pienc,Eixample,19,NaN,NaN,NaN,"private, public, charter","Secondary School, Kindergarten & Primary Schoo...","High School, Kindergarten & Primary School, Ge...",NaN,NaN,111.0,32.475,336,930
6,la Sagrada Família,Eixample,24,NaN,NaN,NaN,"private, public, charter","Kindergarten & Primary School, Music School, F...","Kindergarten & Primary School, Music School, F...",NaN,NaN,111.8,32.700,493,1007
7,la Dreta de l'Eixample,Eixample,75,NaN,NaN,NaN,"public, charter","Secondary School & FP, Kindergarten, Primary &...","High School, General School (with kindergarten...",NaN,NaN,120.9,36.697,211,392


### 1.9 Places Offer

In [73]:
# Dataset Files
places_offer_file = "/Users/mariatormo02/Desktop/TFG-Code/Data/Education Index Data/Oferta_inicial_de_places_escolars_dels_estudis_obligatoris_20240402.csv"

encoded = detect_encoding(places_offer_file)
print(encoded)

# Read CSV
places_offer_df = pd.read_csv(places_offer_file, encoding='utf-8')

utf-8


In [74]:
places_offer_df.columns

Index(['Curs', 'Codi centre', 'Denominació completa', 'Codi naturalesa',
       'Nom naturalesa', 'Codi titularitat', 'Nom titularitat',
       'Codi delegació', 'Nom delegació', 'Codi comarca', 'Nom comarca',
       'Codi municipi_6', 'Nom municipi', 'Codi districte municipal', 'Nom DM',
       'Coordenades_GEO_X', 'Coordenades_GEO_Y', 'Codi ensenyament',
       'Nom ensenyament', 'Nivell', 'Nombre grups', 'Mixt', 'Nombre places',
       'Places ofertades a la preinscripció', 'Georeferència'],
      dtype='object')

In [75]:
places_offer_df = places_offer_df.drop(columns={'Codi naturalesa', 'Nom naturalesa', 'Codi titularitat', 'Codi delegació', 'Codi comarca',
                                                'Codi municipi_6', 'Codi ensenyament', 'Nivell', 'Nombre grups', 'Mixt', 'Georeferència'})

places_offer_df.rename(columns={'Codi centre': 'ID', 'Denominació completa': 'Name', 'Nom titularitat': 'Ownership', 'Nom delegació': 'Delegation Name',
                                'Nom comarca': 'Region Name', 'Nom municipi': 'Town Name', 'Codi districte municipal': 'District ID',
                                'Nom DM': 'District Name', 'Coordenades_GEO_X': 'Longitude', 'Coordenades_GEO_Y': 'Latitude',
                                'Nombre places': 'Number of Places', 'Places ofertades a la preinscripció': 'Places Offered'}, inplace=True)
places_offer_df.head(7)

,Curs,ID,Name,Ownership,Delegation Name,Region Name,Town Name,District ID,District Name,Longitude,Latitude,Nom ensenyament,Number of Places,Places Offered
0,2024/2025,43008419,Institut Camarles,Departament d'Educació,Terres de l'Ebre,Baix Ebre,Camarles,0,NaN,0.658462,40.765611,Educació secundària obligatòria,90,90
1,2024/2025,25001126,Escola Vallmajor - ZER Elaia,Departament d'Educació,Lleida,Garrigues,Bovera,0,NaN,0.638488,41.325428,Educació primària,2,0
2,2024/2025,25001001,Escola Sant Bartomeu - ZER Espernallac,Departament d'Educació,Lleida,Noguera,Bellmunt d'Urgell,0,NaN,0.953486,41.773514,Educació primària,0,0
3,2024/2025,17006769,Institut de Llançà,Departament d'Educació,Girona,Alt Empordà,Llançà,0,NaN,3.153030,42.359094,Educació secundària obligatòria,90,0
4,2024/2025,25004036,Escola L'Estel,Departament d'Educació,Lleida,Segarra,Sant Guim de Freixenet,0,NaN,1.420193,41.657002,Educació primària,25,9
5,2024/2025,43005005,Escola Sant Bernat Calvó,Departament d'Educació,Tarragona,Tarragonès,Vila-seca,0,NaN,1.147857,41.111652,Educació infantil: Parvulari,19,19
6,2024/2025,43001395,Escola Teresa Subirats i Mestre - ZER Montsià,Departament d'Educació,Terres de l'Ebre,Montsià,Mas de Barberans,0,NaN,0.374205,40.736972,Educació primària,5,1


In [76]:
places_offer_df = places_offer_df[places_offer_df['Town Name'] == 'Barcelona']
places_offer_df.head()

,Curs,ID,Name,Ownership,Delegation Name,Region Name,Town Name,District ID,District Name,Longitude,Latitude,Nom ensenyament,Number of Places,Places Offered
1400,2024/2025,8077095,Institut Escola Mirades,Departament d'Educació,Consorci d'Educació de Barcelona,Barcelonès,Barcelona,7,Horta - Guinardó,2.152905,41.426163,Educació infantil: Parvulari,40,0
1787,2024/2025,8054848,Institut Bosc de Montjuïc,Corporacions Locals,Consorci d'Educació de Barcelona,Barcelonès,Barcelona,3,Sants - Montjuïc,2.161387,41.368973,Educació secundària obligatòria,60,60
1808,2024/2025,8043747,Escola Baloo,Departament d'Educació,Consorci d'Educació de Barcelona,Barcelonès,Barcelona,7,Horta - Guinardó,2.140470,41.433634,Educació primària,25,0
1809,2024/2025,8036469,Urgell,Altres Confesions Religioses,Consorci d'Educació de Barcelona,Barcelonès,Barcelona,2,Eixample,2.153131,41.386032,Educació infantil: Parvulari,22,22
1868,2024/2025,8007494,Sagrada Família,Fundacions,Consorci d'Educació de Barcelona,Barcelonès,Barcelona,9,Sant Andreu,2.186580,41.438809,Educació primària,75,0


In [77]:
cleaned_educ_df.head()

,Name,Neighborhood ID,Neighborhood Name,District ID,District Name,ZipCode,Service Type,Simple Service Type,Latitude,Longitude
0,Facultat d'Informació i Mitjans Audiovisuals - UB,18,Sants,3,Sants-Montjuïc,8014,University,University,41.381286,2.138788
1,Institut Fort Pius,5,el Fort Pienc,2,Eixample,8013,Secondary School,High School,41.394359,2.179807
2,Escola Pia de Nostra Senyora,7,la Dreta de l'Eixample,2,Eixample,8007,Secondary School & FP,High School,41.391216,2.167916
3,Institut Salvador Seguí,73,la Verneda i la Pau,10,Sant Martí,8020,FP,FP,41.423094,2.196790
4,Escola de Música Kostka,30,la Salut,6,Gràcia,8024,Music School,Music School,41.414357,2.157240


#### Compare Centres in places dataset and cleaned dataset

In [78]:
print(cleaned_educ_df.shape)
print(places_offer_df.shape)

(1357, 10)
(14849, 14)


In [79]:
# Select School Names with nearest year (appear for diverse years)
reference_year = 2024
# Get first year as int
places_offer_df['Year'] = places_offer_df['Curs'].str.split("/").str[0].astype(int)
rows = []
names_places = set(places_offer_df['Name'])

for name in names_places:
    sub_df = places_offer_df[places_offer_df['Name'] == name]
    sub_df['YearDiff'] = abs(sub_df['Year'] - reference_year)
    nearest_row = sub_df.loc[sub_df['YearDiff'].idxmin()]
    rows.append(nearest_row)


# Step 4: Create a new DataFrame from the filtered rows
filtered_places_offer_df = pd.DataFrame(rows).drop(columns=['Year', 'YearDiff']).reset_index(drop=True)

# Print the filtered DataFrame to verify
places_offer_df = filtered_places_offer_df
places_offer_df = places_offer_df.drop(columns={'Curs', 'ID', 'Delegation Name', 'Region Name', 'Town Name', 'Nom ensenyament'})
places_offer_df.head()

,Name,Ownership,District ID,District Name,Longitude,Latitude,Number of Places,Places Offered
0,Escola Eduard Marquina,Departament d'Educació,10,Sant Martí,2.213974,41.418328,20,20
1,Escola Pia Luz Casanova,Ordes i Congregacions Catòlics,8,Nou Barris,2.175233,41.443117,25,6
2,Escola l'Univers,Departament d'Educació,6,Gràcia,2.162168,41.403126,46,46
3,Escola La Farigola de Vallcarca,Departament d'Educació,6,Gràcia,2.146662,41.413802,25,2
4,Institut Escola Tres Fonts de les Corts,Departament d'Educació,4,Les Corts,2.128661,41.384210,49,0


In [80]:
print(cleaned_educ_df.shape)
print(places_offer_df.shape)

(1357, 10)
(430, 8)


In [81]:
def get_best_match(row, choices, scorer, cutoff=80):
    name = row['Name']
    match, score = process.extractOne(name, choices, scorer=scorer)
    return match if score >= cutoff else None

In [82]:
# Assign Neighborhoods using Cleaned Dataset

# Apply the fuzzy matching function to the second dataset
choices = cleaned_educ_df['Name'].tolist()
places_offer_df['Matched_Name'] = places_offer_df.apply(get_best_match, axis=1, choices=choices, scorer=fuzz.partial_ratio)

# Merge with the neighborhood information using the matched names
places_merged_df = places_offer_df.merge(cleaned_educ_df[['Name', 'Neighborhood Name']],
                             left_on='Matched_Name', right_on='Name', how='left')

# Drop the 'Matched_Name' column as it is no longer needed
places_merged_df.drop(columns=['Matched_Name'], inplace=True)

#  Rename and drop
places_merged_df.rename(columns={'Name_x': 'Original Name', 'Name_y': 'Mapped Name', 'Neighborhood Name_y': 'Neighborhood Name'}, inplace=True)

# Display the merged DataFrame
display(places_merged_df)


,Original Name,Ownership,District ID,District Name,Longitude,Latitude,Number of Places,Places Offered,Mapped Name,Neighborhood Name
0,Escola Eduard Marquina,Departament d'Educació,10,Sant Martí,2.213974,41.418328,20,20,Escola Eduard Marquina,el Besòs i el Maresme
1,Escola Pia Luz Casanova,Ordes i Congregacions Catòlics,8,Nou Barris,2.175233,41.443117,25,6,Escola,Pedralbes
2,Escola l'Univers,Departament d'Educació,6,Gràcia,2.162168,41.403126,46,46,Escola L'Univers,la Vila de Gràcia
3,Escola La Farigola de Vallcarca,Departament d'Educació,6,Gràcia,2.146662,41.413802,25,2,Escola la Farigola de Vallcarca,Vallcarca i els Penitents
4,Institut Escola Tres Fonts de les Corts,Departament d'Educació,4,Les Corts,2.128661,41.384210,49,0,Institut Escola Tres Fonts de les Corts,les Corts
...,...,...,...,...,...,...,...,...,...,...
425,Institut Escola Arts,Departament d'Educació,3,Sants - Montjuïc,2.142304,41.375775,25,0,Escola,Pedralbes
426,Mireia,Societats Mercantils,2,Eixample,2.178722,41.393533,25,10,Centre Educatiu Mireia,el Fort Pienc
427,Institut Josep Pla,Departament d'Educació,8,Nou Barris,2.164321,41.431687,90,4,Institut Josep Pla,el Turó de la Peira
428,Virolai,Societats Mercantils,7,Horta - Guinardó,2.151764,41.418308,90,36,Centre Educatiu Virolai,el Carmel


In [83]:
zones_educ_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 73 entries, 1 to 73
Data columns (total 15 columns):
 #   Column                                       Non-Null Count  Dtype  
---  ------                                       --------------  -----  
 0   Neighborhood Name                            73 non-null     object 
 1   District Name Associated                     73 non-null     object 
 2   Number of Services                           73 non-null     int64  
 3   Distances                                    0 non-null      object 
 4   Density over Places Ratio                    0 non-null      object 
 5   Avg. Students per Teacher Ratio              0 non-null      object 
 6   School Status                                73 non-null     object 
 7   Centers Type                                 73 non-null     object 
 8   Centers Simple Type                          73 non-null     object 
 9   Centers Characteristics                      0 non-null      object 
 10  Avg.

In [84]:
# GROUP BY NEIGHBORHOOD

# Count Schools from dataframe for each neighborhood (add places)
neighborhood_places = places_merged_df.groupby('Neighborhood Name')['Number of Places'].sum().reset_index()

# Rename the 'Number of Places' column in neighborhood_places to avoid conflict
neighborhood_places = neighborhood_places.rename(columns={'Number of Places': 'Total Number of Places'})

# Update the 'Density over Places Ratio' column in zones_educ_df
zones_educ_df = zones_educ_df.merge(neighborhood_places, on='Neighborhood Name', how='left')

# If you need the new column to be named 'Number of Places', you can rename it back
zones_educ_df = zones_educ_df.rename(columns={'Total Number of Places': 'Number of Places'})
zones_educ_df['Density over Places Ratio'] = zones_educ_df['Population Density (hab/ha)']/zones_educ_df['Number of Places']

display(zones_educ_df)

,Neighborhood Name,District Name Associated,Number of Services,Distances,Density over Places Ratio,Avg. Students per Teacher Ratio,School Status,Centers Type,Centers Simple Type,Centers Characteristics,Avg. Dropout Rate,Territorial Socioeconomic Index (IST),Avg. Gini Index (%),Population Density (hab/ha),Net Population Density (hab/residential ha),Number of Places
0,el Raval,Ciutat Vella,45,NaN,1.004684,NaN,"public, charter","Kindergarten & Primary School, University, Kin...","Kindergarten & Primary School, University, Gen...",NaN,NaN,78.2,34.495,429,952,427.0
1,el Barri Gòtic,Ciutat Vella,13,NaN,1.685535,NaN,"public, charter","FP, Kindergarten & Primary School, University,...","FP, Kindergarten & Primary School, University,...",NaN,NaN,75.6,39.133,268,639,159.0
2,la Barceloneta,Ciutat Vella,23,NaN,0.696629,NaN,public,"Secondary School, University, Kindergarten & P...","High School, University, Kindergarten & Primar...",NaN,NaN,95.8,33.636,124,1081,178.0
3,"Sant Pere, Santa Caterina i la Ribera",Ciutat Vella,35,NaN,0.710714,NaN,"public, charter","Kindergarten & Primary School, Secondary Schoo...","Kindergarten & Primary School, High School, Mu...",NaN,NaN,100.8,37.031,199,683,280.0
4,el Fort Pienc,Eixample,19,NaN,1.116279,NaN,"private, public, charter","Secondary School, Kindergarten & Primary Schoo...","High School, Kindergarten & Primary School, Ge...",NaN,NaN,111.0,32.475,336,930,301.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68,Diagonal Mar i el Front Marítim del Poblenou,Sant Martí,7,NaN,1.090000,NaN,"public, charter","Kindergarten, Primary & Secondary School, FP, ...","General School (with kindergarten), FP, Kinder...",NaN,NaN,116.6,31.814,109,556,100.0
69,el Besòs i el Maresme,Sant Martí,13,NaN,0.600567,NaN,"public, charter","Kindergarten & Primary School, Secondary Schoo...","Kindergarten & Primary School, High School, Un...",NaN,NaN,72.4,29.715,212,913,353.0
70,Provençals del Poblenou,Sant Martí,17,NaN,1.237179,NaN,"public, charter","Kindergarten & Primary School, University, Inf...","Kindergarten & Primary School, University, Inf...",NaN,NaN,107.0,29.242,193,1368,156.0
71,Sant Martí de Provençals,Sant Martí,19,NaN,0.579119,NaN,"public, charter","Kindergarten & Primary School, Kindergarten, P...","Kindergarten & Primary School, General School ...",NaN,NaN,102.9,27.361,355,1111,613.0


### 1.10 Center Characteristics

1.10.1 Religious


In [85]:
places_offer_df['Ownership'].unique()

array(["Departament d'Educació", 'Ordes i Congregacions Catòlics',
       'Societats Mercantils', 'Corporacions Locals', 'Fundacions',
       'Cooperatives', 'Associacions', 'Persones Físiques',
       'Altres Confesions Religioses', 'Església Catòlica'], dtype=object)

In [86]:
religious_df = pd.DataFrame(places_offer_df)
religious_df = religious_df.drop(columns={'Latitude', 'Longitude', 'Number of Places', 'Places Offered'})
religious_df = religious_df[religious_df['Ownership'] != "Departament d'Educació"]
religious_df = religious_df[religious_df['Ownership'] != "Corporacions Locals"]
religious_df = religious_df[religious_df['Ownership'] != "Persones Físiques"]
religious_df = religious_df[religious_df['Ownership'] != "Cooperatives"]
religious_df = religious_df[religious_df['Ownership'] != "Associacions"]
religious_df = religious_df[religious_df['Ownership'] != "Altres Confesions Religioses"]

# Exclude Particular Schools from Fundacions Category:
religious_df = religious_df[religious_df['Name'] != 'Mary Ward']
religious_df = religious_df[religious_df['Name'] != 'Decroly']
religious_df = religious_df[religious_df['Name'] != 'Sadako']
religious_df = religious_df[religious_df['Name'] != 'Santísima Trinidad']
religious_df = religious_df[religious_df['Name'] != 'Galí Bellesguard']
religious_df = religious_df[religious_df['Name'] != 'Sopeña Barcelona']
religious_df = religious_df[religious_df['Name'] != 'Frederic Mistral-Tècnic Eulàlia']
religious_df = religious_df[religious_df['Name'] != 'Santa Anna-Eixample']
religious_df = religious_df[religious_df['Name'] != 'Bon Pastor']
religious_df = religious_df[religious_df['Name'] != 'Thau']
religious_df = religious_df[religious_df['Name'] != 'Sant Gregori']
religious_df = religious_df[religious_df['Name'] != 'Santíssima Trinitat']
religious_df = religious_df[religious_df['Name'] != 'El Petit Santa Maria']
religious_df = religious_df[religious_df['Name'] != 'Sagrada Família']
religious_df = religious_df[religious_df['Name'] != 'Voramar']
religious_df = religious_df[religious_df['Name'] != 'Sant Felip Neri']
religious_df = religious_df[religious_df['Name'] != 'Sant Medir']
religious_df = religious_df[religious_df['Name'] != 'Santa Teresa de Lisieux']
religious_df = religious_df[religious_df['Name'] != 'Mare de Déu del Roser Les Planes']
religious_df = religious_df[religious_df['Name'] != 'Peter Pan']
religious_df = religious_df[religious_df['Name'] != 'Joan Pelegrí']
religious_df = religious_df[religious_df['Name'] != "L'Horitzó"]
religious_df = religious_df[religious_df['Name'] != 'Nausica']
religious_df = religious_df[religious_df['Name'] != 'Centre Educatiu Projecte']

# Exclude particular schools of Socetats Mercantils Ownership:
religious_df = religious_df[religious_df['Name'] != 'Virolai']
religious_df = religious_df[religious_df['Name'] != 'Grimm']
religious_df = religious_df[religious_df['Name'] != 'Garbiñe']
religious_df = religious_df[religious_df['Name'] != 'Grey']
religious_df = religious_df[religious_df['Name'] != 'San Rafael']
religious_df = religious_df[religious_df['Name'] != 'Mireia']
religious_df = religious_df[religious_df['Name'] != 'Súnion']
religious_df = religious_df[religious_df['Name'] != 'Joan Roca Guipúzcoa']
religious_df = religious_df[religious_df['Name'] != 'Sant Ferran']
religious_df = religious_df[religious_df['Name'] != 'Reina Elisenda']
religious_df = religious_df[religious_df['Name'] != 'Molina']
religious_df = religious_df[religious_df['Name'] != 'Lloret']
religious_df = religious_df[religious_df['Name'] != 'Laia']
religious_df = religious_df[religious_df['Name'] != 'Aloma']
religious_df = religious_df[religious_df['Name'] != 'Ramon Llull']
religious_df = religious_df[religious_df['Name'] != 'Alfageme']
religious_df = religious_df[religious_df['Name'] != 'Mare de Déu de Núria']
religious_df = religious_df[religious_df['Name'] != 'Sant Francesc Xavier']
religious_df = religious_df[religious_df['Name'] != 'Lleó XIII']
religious_df = religious_df[religious_df['Name'] != 'Palcam']
religious_df = religious_df[religious_df['Name'] != 'Closa']
religious_df = religious_df[religious_df['Name'] != 'Roca']
religious_df = religious_df[religious_df['Name'] != 'Monlau']
religious_df = religious_df[religious_df['Name'] != 'IPSI']
religious_df = religious_df[religious_df['Name'] != "L'Esclop-Passeig"]
religious_df = religious_df[religious_df['Name'] != 'John Talabot']
religious_df = religious_df[religious_df['Name'] != 'Camí']
religious_df = religious_df[religious_df['Name'] != 'Sant Marc de Sarrià']
religious_df = religious_df[religious_df['Name'] != 'Castro de la Peña']
religious_df = religious_df[religious_df['Name'] != 'Joan Roca 1953']
religious_df = religious_df[religious_df['Name'] != 'Miró']
religious_df = religious_df[religious_df['Name'] != 'Sant Lluís']
religious_df = religious_df[religious_df['Name'] != 'Sant Pau']

In [87]:
religious_df.head()

,Name,Ownership,District ID,District Name,Matched_Name
1,Escola Pia Luz Casanova,Ordes i Congregacions Catòlics,8,Nou Barris,Escola
10,L'Esperança,Fundacions,9,Sant Andreu,Centre Educatiu FEP L'Esperança
13,Padre Damián Sagrados Corazones,Ordes i Congregacions Catòlics,5,Sarrià - Sant Gervasi,Centre Educatiu Padre Damián Sagrados Corazones
17,Jesús Maria,Ordes i Congregacions Catòlics,3,Sants - Montjuïc,Centre Educatiu Jesús Maria
22,Regina Carmeli,Ordes i Congregacions Catòlics,7,Horta - Guinardó,Centre Educatiu Regina Carmeli


In [88]:
religious_df.reset_index()
religious_df.head()

,Name,Ownership,District ID,District Name,Matched_Name
1,Escola Pia Luz Casanova,Ordes i Congregacions Catòlics,8,Nou Barris,Escola
10,L'Esperança,Fundacions,9,Sant Andreu,Centre Educatiu FEP L'Esperança
13,Padre Damián Sagrados Corazones,Ordes i Congregacions Catòlics,5,Sarrià - Sant Gervasi,Centre Educatiu Padre Damián Sagrados Corazones
17,Jesús Maria,Ordes i Congregacions Catòlics,3,Sants - Montjuïc,Centre Educatiu Jesús Maria
22,Regina Carmeli,Ordes i Congregacions Catòlics,7,Horta - Guinardó,Centre Educatiu Regina Carmeli


#### Add neighborhood ID corresponding to each school in dataset

In [89]:
schools_neighborhood_df = cleaned_educ_df.drop(columns={'Neighborhood ID', 'ZipCode', 'Service Type', 'Simple Service Type', 'Latitude', 'Longitude'})
schools_neighborhood_df.rename(columns={'District ID': 'District ID 2', 'District Name': 'District Name 2'}, inplace=True)
schools_neighborhood_df.head()

,Name,Neighborhood Name,District ID 2,District Name 2
0,Facultat d'Informació i Mitjans Audiovisuals - UB,Sants,3,Sants-Montjuïc
1,Institut Fort Pius,el Fort Pienc,2,Eixample
2,Escola Pia de Nostra Senyora,la Dreta de l'Eixample,2,Eixample
3,Institut Salvador Seguí,la Verneda i la Pau,10,Sant Martí
4,Escola de Música Kostka,la Salut,6,Gràcia


In [90]:
# Apply the fuzzy matching function to the second dataset
choices = schools_neighborhood_df['Name'].tolist()
religious_df['Matched_Name'] = religious_df.apply(get_best_match, axis=1, choices=choices, scorer=fuzz.partial_ratio)

# Merge with the neighborhood information using the matched names
merged_df = religious_df.merge(schools_neighborhood_df[['Name', 'Neighborhood Name']],
                             left_on='Matched_Name', right_on='Name', how='left')

# Drop the 'Matched_Name' column as it is no longer needed
merged_df.drop(columns=['Matched_Name'], inplace=True)

#  Rename and drop
#merged_df.drop(columns={'Neighborhood Name_x'}, inplace=True)
merged_df.rename(columns={'Name_x': 'Original Name', 'Name_y': 'Mapped Name', 'Neighborhood Name_y': 'Neighborhood Name'}, inplace=True)

# Display the merged DataFrame
display(merged_df)
religious_df = merged_df

,Original Name,Ownership,District ID,District Name,Mapped Name,Neighborhood Name
0,Escola Pia Luz Casanova,Ordes i Congregacions Catòlics,8,Nou Barris,Escola,Pedralbes
1,L'Esperança,Fundacions,9,Sant Andreu,Centre Educatiu FEP L'Esperança,Baró de Viver
2,Padre Damián Sagrados Corazones,Ordes i Congregacions Catòlics,5,Sarrià - Sant Gervasi,Centre Educatiu Padre Damián Sagrados Corazones,Sarrià
3,Jesús Maria,Ordes i Congregacions Catòlics,3,Sants - Montjuïc,Centre Educatiu Jesús Maria,Sant Andreu
4,Regina Carmeli,Ordes i Congregacions Catòlics,7,Horta - Guinardó,Centre Educatiu Regina Carmeli,Horta
...,...,...,...,...,...,...
92,Jesuïtes Casp - Sagrat Cor de Jesús,Fundacions,2,Eixample,Centre Educatiu Jesuïtes Casp - Sagrat Cor de ...,la Dreta de l'Eixample
93,Sant Joan Bosco,Ordes i Congregacions Catòlics,7,Horta - Guinardó,Centre Educatiu Sant Joan Bosco,la Vall d'Hebron
94,Sagrat Cor-Sarrià,Fundacions,5,Sarrià - Sant Gervasi,Centre Educatiu Sagrat Cor-Sarrià,Sarrià
95,FEP Santa Teresa de Lisieux,Fundacions,4,Les Corts,Centre Educatiu FEP Santa Teresa de Lisieux,les Corts


In [91]:
# GROUP BY NEIGHBORHOOD

# Count Schools from dataframe (religious) for each neighborhood
neighborhood_counts = religious_df.groupby('Neighborhood Name').size()
#print(neighborhood_counts)

# Find neighborhoods with at least one school
neighborhoods_with_schools = neighborhood_counts[neighborhood_counts > 0].index

# Update the 'Centers Characteristics' column in zones_educ_df
zones_educ_df['Centers Characteristics'] = "non-religious"
zones_educ_df.loc[zones_educ_df['Neighborhood Name'].isin(neighborhoods_with_schools), 'Centers Characteristics'] = 'religious and non-religious'

display(zones_educ_df)

,Neighborhood Name,District Name Associated,Number of Services,Distances,Density over Places Ratio,Avg. Students per Teacher Ratio,School Status,Centers Type,Centers Simple Type,Centers Characteristics,Avg. Dropout Rate,Territorial Socioeconomic Index (IST),Avg. Gini Index (%),Population Density (hab/ha),Net Population Density (hab/residential ha),Number of Places
0,el Raval,Ciutat Vella,45,NaN,1.004684,NaN,"public, charter","Kindergarten & Primary School, University, Kin...","Kindergarten & Primary School, University, Gen...",religious and non-religious,NaN,78.2,34.495,429,952,427.0
1,el Barri Gòtic,Ciutat Vella,13,NaN,1.685535,NaN,"public, charter","FP, Kindergarten & Primary School, University,...","FP, Kindergarten & Primary School, University,...",non-religious,NaN,75.6,39.133,268,639,159.0
2,la Barceloneta,Ciutat Vella,23,NaN,0.696629,NaN,public,"Secondary School, University, Kindergarten & P...","High School, University, Kindergarten & Primar...",non-religious,NaN,95.8,33.636,124,1081,178.0
3,"Sant Pere, Santa Caterina i la Ribera",Ciutat Vella,35,NaN,0.710714,NaN,"public, charter","Kindergarten & Primary School, Secondary Schoo...","Kindergarten & Primary School, High School, Mu...",religious and non-religious,NaN,100.8,37.031,199,683,280.0
4,el Fort Pienc,Eixample,19,NaN,1.116279,NaN,"private, public, charter","Secondary School, Kindergarten & Primary Schoo...","High School, Kindergarten & Primary School, Ge...",non-religious,NaN,111.0,32.475,336,930,301.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68,Diagonal Mar i el Front Marítim del Poblenou,Sant Martí,7,NaN,1.090000,NaN,"public, charter","Kindergarten, Primary & Secondary School, FP, ...","General School (with kindergarten), FP, Kinder...",non-religious,NaN,116.6,31.814,109,556,100.0
69,el Besòs i el Maresme,Sant Martí,13,NaN,0.600567,NaN,"public, charter","Kindergarten & Primary School, Secondary Schoo...","Kindergarten & Primary School, High School, Un...",religious and non-religious,NaN,72.4,29.715,212,913,353.0
70,Provençals del Poblenou,Sant Martí,17,NaN,1.237179,NaN,"public, charter","Kindergarten & Primary School, University, Inf...","Kindergarten & Primary School, University, Inf...",non-religious,NaN,107.0,29.242,193,1368,156.0
71,Sant Martí de Provençals,Sant Martí,19,NaN,0.579119,NaN,"public, charter","Kindergarten & Primary School, Kindergarten, P...","Kindergarten & Primary School, General School ...",religious and non-religious,NaN,102.9,27.361,355,1111,613.0


1.10.2 International (specify language?)

### 1.11. Students per Teacher Ratio

In [92]:
# Dataset Files
students_file = "/Users/mariatormo02/Desktop/TFG-Code/Data/Education Index Data/Alumnes_matriculats_per_ensenyament_i_unitats_dels_centres_docents_20240402.csv"
teachers_file = "/Users/mariatormo02/Desktop/TFG-Code/Data/Education Index Data/Personal_docent_en_centres_p_blics_titularitat_del_Departament_d_Educaci__20240402.csv"

#encoded = detect_encoding(students_file)
#print(encoded)

# Read CSV
students_df = pd.read_csv(students_file, encoding='utf-8', nrows=44463)
teachers_df = pd.read_csv(teachers_file, encoding='utf-8')

STUDENTS

In [93]:
students_df.head()

,Curs,Any,Codi centre,Denominació completa,Codi naturalesa,Nom naturalesa,Codi titularitat,Nom titularitat,Codi delegació,Nom delegació,...,Nom ensenyament,Nivell,Matrícula concertada,Modalitat,Període matrícula,Matrícules. Total,Matrícules. Dones,Matrícules. Homes,Unitats,Georeferència
0,2022/2023,2022,43960064,PTT Segur de Calafell,1,Públic,99,No informat,208,Barcelona Comarques,...,AUXILIAR DE VIVERS I JARDINS,1,NO,PRESENCIAL,ANUAL,11,2,9,0.0,NaN
1,2022/2023,2022,43960066,PTT Constantí,1,Públic,99,No informat,143,Tarragona,...,AUXILIAR DE COMERCIALITZACIÓ DE PRODUCTES I LO...,1,NO,PRESENCIAL,ANUAL,15,3,12,0.0,NaN
2,2022/2023,2022,43012629,Aula Magna,2,Privat,26,Societats Mercantils,143,Tarragona,...,CFPS - EDUCACIÓ INFANTIL,2,NO,PRESENCIAL,ANUAL,20,17,3,0.0,NaN
3,2022/2023,2022,43012629,Aula Magna,2,Privat,26,Societats Mercantils,143,Tarragona,...,CFPS - INTEGRACIÓ SOCIAL,1,NO,NO PRESENCIAL,ANUAL,10,9,1,0.0,NaN
4,2022/2023,2022,43012629,Aula Magna,2,Privat,26,Societats Mercantils,143,Tarragona,...,CFPS - INTEGRACIÓ SOCIAL,2,NO,NO PRESENCIAL,ANUAL,12,10,2,0.0,NaN


In [94]:
# Select Most Recent Year
students_df = students_df[students_df['Any'] == 2022]
students_df = students_df[students_df['Nom comarca'] == 'Barcelonès']

In [95]:

#students_df = students_df[students_df['Nom municipi'] == 'Barcelona']
# Drop Unnecessary columns
cols_to_drop_stud = ['Curs', 'Any', 'Codi centre', 'Codi naturalesa', 'Nom naturalesa', 'Codi titularitat', 'Nom titularitat', 'Codi delegació', 'Nom delegació', 'Codi comarca', 'Nom comarca',
                     'Codi municipi_5', 'Codi municipi_6', 'Nom municipi', 'Nom districte municipal', 'Coordenades UTM X', 'Coordenades UTM Y',
                     'Coordenades GEO X', 'Coordenades GEO Y', 'Codi estudis', 'Nom estudis', 'Temàtica', 'Grau', 'Codi ensenyament', 'Nom ensenyament',
                     'Nivell', 'Matrícula concertada', 'Modalitat', 'Període matrícula', 'Georeferència', 'Unitats', 'Matrícules. Dones', 'Matrícules. Homes']

students_df.drop(columns=cols_to_drop_stud, inplace=True)
students_df.rename(columns={'Denominació completa': 'Name', 'Codi districte municipal': 'District ID', 'Matrícules. Total': 'Number of Students'}, inplace=True)
students_df.reset_index()
students_df.head(8)

,Name,District ID,Number of Students
4690,Institut de Logística de Barcelona,1,32
4691,Institut de Logística de Barcelona,1,26
4692,Institut de Logística de Barcelona,1,32
4693,Institut de Logística de Barcelona,1,55
4694,Institut de Logística de Barcelona,1,63
4695,Institut de Logística de Barcelona,1,28
4696,Institut de Logística de Barcelona,1,32
4697,Institut de Logística de Barcelona,1,30


In [96]:
# Add up students per center Name (different services type: primary, secondary, etc.)
center_students_df = students_df.groupby(['Name', 'District ID'], as_index=False)['Number of Students'].sum().reset_index()
students_df = center_students_df
students_df.head()

,index,Name,District ID,Number of Students
0,0,19@,5,128
1,1,AA Eurofitness Edu,2,241
2,2,AAFE Olimpia,10,79
3,3,AF Centre d'estudis,1,189
4,4,AFA Ciutat Meridiana,8,360


In [97]:
# Function to get the best match with a minimum score threshold and district matching
def get_best_match(row, choices, scorer, threshold=95):
    # Filter choices by matching district
    district_choices = [choice for choice in choices if choice[1] == row['District ID']]

    if not district_choices:
        return None

    best_match, best_score = process.extractOne(row['Name'], [choice[0] for choice in district_choices], scorer=scorer)

    if best_score >= threshold:
        return best_match
    else:
        return None

# Assign Neighborhoods using Cleaned Dataset

# Assuming 'District Name' column exists in both datasets
# Prepare the choices list with tuples of (Name, District)
choices = list(cleaned_educ_df[['Name', 'District ID']].itertuples(index=False, name=None))

# Apply the fuzzy matching function to the second dataset
students_df['Matched_Name'] = students_df.apply(get_best_match, axis=1, choices=choices, scorer=fuzz.partial_ratio)

# Filter out rows where no good match was found
students_df = students_df[students_df['Matched_Name'].notna()]

# Merge with the neighborhood information using the matched names
students_merged_df = students_df.merge(cleaned_educ_df[['Name', 'Neighborhood Name']],
                                   left_on='Matched_Name', right_on='Name', how='left')

# Drop the 'Matched_Name' column as it is no longer needed
students_merged_df.drop(columns=['Matched_Name'], inplace=True)

# Rename and drop columns
students_merged_df.rename(columns={'Name_x': 'Original Name', 'Name_y': 'Mapped Name', 'Neighborhood Name_y': 'Neighborhood Name'}, inplace=True)

# Display the merged DataFrame
display(students_merged_df)
students_df = students_merged_df

,index,Original Name,District ID,Number of Students,Mapped Name,Neighborhood Name
0,0,19@,5,128,Centre de Formació 19@,Sant Gervasi - la Bonanova
1,10,Abat Oliba-Spínola,7,820,Centre Educatiu Abat Oliba-Spínola,el Baix Guinardó
2,11,Acis,6,18,Escola d'Adults Acis,la Salut
3,13,Adela de Trenquelleón,10,680,Centre Educatiu Adela de Trenquelleón,Sant Martí de Provençals
4,16,Alfageme,3,190,Centre Educatiu Alfageme,la Marina de Port
...,...,...,...,...,...,...
521,1121,Voramar,10,638,Centre Educatiu Voramar,el Poblenou
522,1122,Waldorf Steiner,7,4,Parvulari Waldorf Steiner,la Font d'en Fargues
523,1125,Xumets,3,32,Llar d'Infants Xumets,la Marina de Port
524,1126,Zurich,4,302,Centre Educatiu Zurich,Pedralbes


In [98]:
# SUM STUDENTS BY NEIGHBORHOOD
neighborhood_students = students_df.groupby('Neighborhood Name')['Number of Students'].sum().reset_index()


# Update the 'Number of Students' column in zones_educ_df
zones_educ_df = zones_educ_df.merge(neighborhood_students, on='Neighborhood Name', how='left')

display(zones_educ_df)

,Neighborhood Name,District Name Associated,Number of Services,Distances,Density over Places Ratio,Avg. Students per Teacher Ratio,School Status,Centers Type,Centers Simple Type,Centers Characteristics,Avg. Dropout Rate,Territorial Socioeconomic Index (IST),Avg. Gini Index (%),Population Density (hab/ha),Net Population Density (hab/residential ha),Number of Places,Number of Students
0,el Raval,Ciutat Vella,45,NaN,1.004684,NaN,"public, charter","Kindergarten & Primary School, University, Kin...","Kindergarten & Primary School, University, Gen...",religious and non-religious,NaN,78.2,34.495,429,952,427.0,9128.0
1,el Barri Gòtic,Ciutat Vella,13,NaN,1.685535,NaN,"public, charter","FP, Kindergarten & Primary School, University,...","FP, Kindergarten & Primary School, University,...",non-religious,NaN,75.6,39.133,268,639,159.0,1066.0
2,la Barceloneta,Ciutat Vella,23,NaN,0.696629,NaN,public,"Secondary School, University, Kindergarten & P...","High School, University, Kindergarten & Primar...",non-religious,NaN,95.8,33.636,124,1081,178.0,1554.0
3,"Sant Pere, Santa Caterina i la Ribera",Ciutat Vella,35,NaN,0.710714,NaN,"public, charter","Kindergarten & Primary School, Secondary Schoo...","Kindergarten & Primary School, High School, Mu...",religious and non-religious,NaN,100.8,37.031,199,683,280.0,4360.0
4,el Fort Pienc,Eixample,19,NaN,1.116279,NaN,"private, public, charter","Secondary School, Kindergarten & Primary Schoo...","High School, Kindergarten & Primary School, Ge...",non-religious,NaN,111.0,32.475,336,930,301.0,2817.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68,Diagonal Mar i el Front Marítim del Poblenou,Sant Martí,7,NaN,1.090000,NaN,"public, charter","Kindergarten, Primary & Secondary School, FP, ...","General School (with kindergarten), FP, Kinder...",non-religious,NaN,116.6,31.814,109,556,100.0,2628.0
69,el Besòs i el Maresme,Sant Martí,13,NaN,0.600567,NaN,"public, charter","Kindergarten & Primary School, Secondary Schoo...","Kindergarten & Primary School, High School, Un...",religious and non-religious,NaN,72.4,29.715,212,913,353.0,5363.0
70,Provençals del Poblenou,Sant Martí,17,NaN,1.237179,NaN,"public, charter","Kindergarten & Primary School, University, Inf...","Kindergarten & Primary School, University, Inf...",non-religious,NaN,107.0,29.242,193,1368,156.0,1972.0
71,Sant Martí de Provençals,Sant Martí,19,NaN,0.579119,NaN,"public, charter","Kindergarten & Primary School, Kindergarten, P...","Kindergarten & Primary School, General School ...",religious and non-religious,NaN,102.9,27.361,355,1111,613.0,4287.0


TEACHERS

In [99]:
#teachers_df['Nom municipi'].unique()

In [100]:
#display(teachers_df)
teachers_df = teachers_df[teachers_df['Curs'] == '2023/2024']
teachers_df = teachers_df[teachers_df['Nom municipi'] == 'BARCELONA']

# Get rid of useless columns and rename
teachers_df.drop(columns=['Curs', 'Codi centre', 'Codi municipi_5', 'Codi municipi_6', 'Nom municipi', 'Dones', 'Homes'], inplace=True)
teachers_df.rename(columns={'Denominació completa': 'Name', 'Total': 'Number of Teachers'}, inplace=True)
teachers_df.head()

,Name,Number of Teachers
1026,INSTITUT DE TÈCNIQUES AUDIOVISUALS I DE L'ESPE...,26
1027,INSTITUT TIC DE BARCELONA,48
1028,ESCOLA SEGRE,18
1029,INSTITUT ESCOLA SANT FELIP NERI,28
1034,INSTITUT DE LOGÍSTICA DE BARCELONA,47


In [101]:
# Add up students per center Name (different services type: primary, secondary, etc.)
center_teachers_df = teachers_df.groupby('Name')['Number of Teachers'].sum().reset_index()
teachers_df = center_teachers_df
teachers_df.head()

,Name,Number of Teachers
0,AFA CIUTAT MERIDIANA,7
1,CEE CONCHA ESPINA,20
2,CEE FOLCH I CAMARASA,11
3,CEE JOSEP PLA,15
4,CEE SANT JOAN DE LA CREU,34


In [ ]:
def get_best_match(row, choices, scorer, cutoff=80):
    name = row['Name']
    match, score = process.extractOne(name, choices, scorer=scorer)
    return match if score >= cutoff else None

# Assign Neighborhoods using Cleaned Dataset

# Apply the fuzzy matching function to the second dataset
choices = cleaned_educ_df['Name'].tolist()
teachers_df['Matched_Name'] = teachers_df.apply(get_best_match, axis=1, choices=choices, scorer=fuzz.partial_ratio)

# Merge with the neighborhood information using the matched names
teachers_merged_df = teachers_df.merge(cleaned_educ_df[['Name', 'Neighborhood Name']],
                             left_on='Matched_Name', right_on='Name', how='left')

# Drop the 'Matched_Name' column as it is no longer needed
teachers_merged_df.drop(columns=['Matched_Name'], inplace=True)

#  Rename and drop
#merged_df.drop(columns={'Neighborhood Name_x'}, inplace=True)
teachers_merged_df.rename(columns={'Name_x': 'Original Name', 'Name_y': 'Mapped Name', 'Neighborhood Name_y': 'Neighborhood Name'}, inplace=True)

# Display the merged DataFrame
display(teachers_merged_df)
teachers_df = teachers_merged_df

In [ ]:
# SUM STUDENTS BY NEIGHBORHOOD
neighborhood_teachers = teachers_df.groupby('Neighborhood Name')['Number of Teachers'].sum().reset_index()
display(neighborhood_teachers)

# Update the 'Density over Places Ratio' column in zones_educ_df
zones_educ_df = zones_educ_df.merge(neighborhood_teachers, on='Neighborhood Name', how='left')
zones_educ_df['Avg. Students per Teacher Ratio'] = zones_educ_df['Number of Students']/zones_educ_df['Number of Teachers']
display(zones_educ_df)

### 1.12. Avg. Dropout Rate